In [1]:
import os

curr_folder = os.getcwd() + "/"
datasets_folder = curr_folder[:-6] + "Datasets/"
datasets_folder

'/home/ipsilva/Repositories/dynamic-projections/Datasets/'

In [2]:
datasets_subfolders = []
for i in os.walk(datasets_folder):
    datasets_subfolders.append(i[0])

datasets_subfolders = datasets_subfolders[1:]
datasets_subfolders

['/home/ipsilva/Repositories/dynamic-projections/Datasets/gaussians',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/quickdraw',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/sorts',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/walk',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/nnset',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/fashion',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cifar10cnn',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/esc50',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/qtables',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cartolastd']

In [3]:
dataset_number = 4
selected_dataset = datasets_subfolders[dataset_number]
selected_dataset

'/home/ipsilva/Repositories/dynamic-projections/Datasets/nnset'

In [4]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir(selected_dataset) if isfile(join(selected_dataset, f))]
files[:5]

['nnset-0.csv', 'nnset-10.csv', 'nnset-25.csv', 'nnset-3.csv', 'nnset-27.csv']

In [5]:
from sklearn.preprocessing import MinMaxScaler
from itertools import product
import pandas as pd
import numpy as np

df = pd.DataFrame()

obs_per_timestep = []
for file in files:
    file_loc = selected_dataset + "/" + file
    ndf = pd.read_csv(file_loc)
    df = df.append(ndf)
    obs_per_timestep.append(ndf.shape[0])
    
median_obs_per_timestep = np.median(obs_per_timestep)
dev_obs_per_timestep = np.std(obs_per_timestep)

min_max_scaler = MinMaxScaler()
df = pd.DataFrame(min_max_scaler.fit_transform(df.drop(columns=df.columns[0], axis=1)))
print(df.shape, df.head())

(2400, 8070)        0         1         2         3         4         5         6     \
0  0.322770  0.279649  0.335847  0.424952  0.490591  0.506529  0.524146   
1  0.358230  0.390066  0.447455  0.507796  0.556292  0.554613  0.557266   
2  0.392771  0.216740  0.303905  0.416151  0.486390  0.503915  0.521783   
3  0.512320  0.407074  0.379818  0.449019  0.506695  0.518385  0.534443   
4  0.665977  0.540487  0.528186  0.593423  0.641495  0.640467  0.645731   

       7         8         9     ...      8060      8061      8062      8063  \
0  0.549136  0.542073  0.551281  ...  0.437468  0.430445  0.431587  0.434024   
1  0.566607  0.548507  0.546772  ...  0.373889  0.358949  0.357158  0.353526   
2  0.544098  0.537360  0.544763  ...  0.396023  0.385041  0.386434  0.386826   
3  0.555051  0.546087  0.552993  ...  0.376948  0.365778  0.368775  0.370560   
4  0.656747  0.635437  0.629814  ...  0.328294  0.315302  0.317730  0.314419   

       8064      8065      8066      8067      8068    

In [6]:
from sklearn.decomposition import PCA

def intrisic_dim(dataframe, target_variance):
    pca = PCA()

    principalComponents = pca.fit_transform(df)

    list_variance = pca.explained_variance_ratio_

    final_var = 0
    dims = 0
    for var in list_variance:
        final_var += var
        dims += 1
        if final_var > target_variance:
            break
    return dims, final_var, principalComponents[:dims]

In [7]:
target_variance = 0.9

(dims, final_var, components) = intrisic_dim(df, target_variance)

print("Total time-steps:", len(obs_per_timestep))
print("Registers per time-step (median):", median_obs_per_timestep)
print("Registers per time-step (deviation):", dev_obs_per_timestep)
print("Total dimensions:", df.shape[1])
print("Intrinsic dimensions:", dims)
print("Total variance:", final_var)

Total time-steps: 30
Registers per time-step (median): 80.0
Registers per time-step (deviation): 0.0
Total dimensions: 8070
Intrinsic dimensions: 178
Total variance: 0.9004695907106401
